In [1]:
import torch
import gradientUtils as gu
# Define a threshold for early stopping
threshold = 1e-6
circle_center = torch.tensor([0.0, 0.0])  # Example circle center
radius = 1.0  # Example radius


In [2]:
def autograd(s_i, s_j, s_k, X_g, Y_g, destination):
    # Define an optimizer
    optimizer = torch.optim.Adam([s_i.x, s_i.y, s_j.x, s_j.y, s_k.x, s_k.y], lr=0.01)
    # Previous loss
    prev_loss = float("inf")
    # Training loop
    for epoch in range(100000):
        # Zero the gradients
        optimizer.zero_grad()

        # Compute the vertex
        X, Y = gu.compute_vertex(s_i, s_j, s_k)
        # Compute the vloss
        vloss = (X - X_g) ** 2 + (Y - Y_g) ** 2

        #Compute midpoint_loss
        midpoint_loss = gu.midpoint_loss(s_i, s_j, s_k, circle_center, radius)

        lambda_midpoint = 0.5  
        loss = vloss + lambda_midpoint * midpoint_loss
        # Early stopping condition
        if abs(prev_loss - loss.item()) < threshold:
            print(f"Stopping early at epoch {epoch} due to minimal loss change")
            gu.plot_and_save(
                epoch, s_i, s_j, s_k, circle_center, radius, X_g, Y_g, destination
            )
            break

        prev_loss = loss.item()
        # Backpropagate the error
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Print the loss
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
            gu.plot_and_save(
                epoch, s_i, s_j, s_k, circle_center, radius, X_g, Y_g, destination
            )

    # Print the final site coordinates
    print(f"Final site coordinates:")
    print(f"s_i: ({s_i.x.item()}, {s_i.y.item()})")
    print(f"s_j: ({s_j.x.item()}, {s_j.y.item()})")
    print(f"s_k: ({s_k.x.item()}, {s_k.y.item()})")

    # Compute the final vertex
    final_X, final_Y = gu.compute_vertex(s_i, s_j, s_k)

    # Print the final vertex coordinates
    print(f"Final vertex coordinates: ({final_X.item()}, {final_Y.item()})")
    return s_i, s_j, s_k

In [3]:

# Initialize the sites
s_k = gu.Site(-1.0, -1.0)
s_i = gu.Site(2.0, -2.0)
s_j = gu.Site(-1.50, -2.750)
destination = "images/autograd/bisec_optim/"

# Compute the vertex
X, Y = gu.compute_vertex(s_i, s_j, s_k)
print(f"Initial vertex coordinates: ({X.item()}, {Y.item()})")
# Compute the closest point on the circle to the initial vertex
X_g, Y_g = gu.compute_closest_point_on_circle(X, Y , circle_center, radius)    

s_i, s_j, s_k = autograd(s_i, s_j, s_k, X_g, Y_g, destination)


Initial vertex coordinates: (0.23369565606117249, -2.29891300201416)
Epoch 0, Loss: 21.093338012695312
Epoch 100, Loss: 1.8510757684707642
Epoch 200, Loss: 0.29557451605796814
Epoch 300, Loss: 0.06946959346532822
Epoch 400, Loss: 0.018845420330762863
Epoch 500, Loss: 0.005002234131097794
Epoch 600, Loss: 0.0011734444415196776
Epoch 700, Loss: 0.0002355894393986091
Stopping early at epoch 784 due to minimal loss change
Final site coordinates:
s_i: (0.9681896567344666, -0.5037286877632141)
s_j: (-0.7638256549835205, -1.4955841302871704)
s_k: (-0.3962525725364685, -0.13009096682071686)
Final vertex coordinates: (0.1000332459807396, -0.9959040880203247)
